In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load Shakespeare's text
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


In [ ]:
# Tokenization
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences([text])[0]

In [ ]:
# Create input-output pairs
input_sequences = []
for i in range(1, len(sequences)):
    n_gram_sequence = sequences[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
# Pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Split into predictors and label
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# One-hot encode the labels
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_lstm_model(total_words, max_sequence_len):
    model = Sequential([
        Embedding(total_words, 100, input_length=max_sequence_len-1),
        LSTM(150, return_sequences=False),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

lstm_model = build_lstm_model(total_words, max_sequence_len)
lstm_model.summary()

In [ ]:
from tensorflow.keras.layers import GRU

def build_gru_model(total_words, max_sequence_len):
    model = Sequential([
        Embedding(total_words, 100, input_length=max_sequence_len-1),
        GRU(150, return_sequences=False),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

gru_model = build_gru_model(total_words, max_sequence_len)
gru_model.summary()

In [ ]:
# Train LSTM Model
lstm_history = lstm_model.fit(X, y, epochs=50, verbose=1)

# Train GRU Model
gru_history = gru_model.fit(X, y, epochs=50, verbose=1)

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=1)[0]
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [ ]:
# Generate text using LSTM
lstm_generated_text = generate_text("shall i compare thee", 50, lstm_model, max_sequence_len)
print("LSTM Generated Text:\n", lstm_generated_text)

# Generate text using GRU
gru_generated_text = generate_text("shall i compare thee", 50, gru_model, max_sequence_len)
print("GRU Generated Text:\n", gru_generated_text)

In [ ]:
import matplotlib.pyplot as plt

# Plot training loss
plt.plot(lstm_history.history['loss'], label='LSTM Loss')
plt.plot(gru_history.history['loss'], label='GRU Loss')
plt.title('Training Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Plot training accuracy
plt.plot(lstm_history.history['accuracy'], label='LSTM Accuracy')
plt.plot(gru_history.history['accuracy'], label='GRU Accuracy')
plt.title('Training Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()